## Hands-On Tabular Workshop: Wine classifier with MLFlow, MinIO, and Seldon

In this workshop, we will run an ML Flow model, save the artefacts to MinIO, and create a v2 protocol SeldonDeployment via the SDK

We will follow these steps:
1. Create environment, install libraries, import methods
2. Run experiment and train model in MLFlow
3. Conda pack the environment
4. Push artefacts to MinIO
5. Deploy via the SDK
6. [Optional] Add model metadata via the SDK
7. [Optional] Create a new MLFlow run with different hyperparametes, and deploy a canary model
8. [Optional] Train and deploy a drift detector
9. [Optional] Train and deploy an explainer

### 1. Create environment, install libraries, import methods

In [1]:
%%sh
conda env list

# conda environments:
#
base                  *  /home/andrew/.asdf/installs/python/miniconda3-4.7.12
envtest                  /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/envtest
mlflow                   /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/mlflow
myenv                    /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/myenv
nngroup                  /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/nngroup
python3.8-mlflow-example     /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/python3.8-mlflow-example
rit                      /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/rit
wine                     /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/wine
                         /home/andrew/seldon/core-tutorials/iris-explainer-poetry/venv



In [2]:
!pip install seldon_deploy_sdk
!pip install minio
!pip install conda-pack
!pip install mlserver
!pip install mlserver-mlflow

In [3]:
from seldon_deploy_sdk import Configuration, PredictApi, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException
from minio import Minio
import pandas as pd
import json
import requests
import os
import glob

### 2. Run experiment and train model in MLFlow
For our example, we will use the elastic net wine example from [MLflow's tutorial](https://github.com/mlflow/mlflow/tree/master/examples/sklearn_elasticnet_wine).

Let's load the data to see what's inside:

In [19]:
data = pd.read_csv("wine-quality.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [17]:
# Wine Quality Sample
def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [18]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.793164022927685
  MAE: 0.6271946374319586
  R2: 0.10862644997792625


In [13]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7336400911821402
  MAE: 0.5643841279275428
  R2: 0.23739466063584158


In [15]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984665
  R2: 0.2799376066653344


Each of these models can actually be found on the `mlruns` folder:

In [24]:
!tree -L 1 mlruns/0

mlruns/0
├── 280a5311590b4c5cb10b052cc6617819
├── 3638146d4e1449d3b133ad4d1484e357
├── 3fed2065421e403287d622d769e246ef
├── 7b1acdcccf1c4fc28934865008a6319c
├── 8d944bf6f329461dba5e8fce94b3a02f
├── ab7b583a5ead4aec806d8c3c0d75fd55
├── d3136a7c576544c2bc136450ac0a51ac
└── meta.yaml

7 directories, 1 file


Inside each of these folders, MLflow stores the parameters we used to train our model, any metric we logged during training, and a snapshot of our model. If we look into one of them, we can see the following structure:

In [25]:
!tree mlruns/0/$(ls mlruns/0 | head -1)

mlruns/0/280a5311590b4c5cb10b052cc6617819
├── artifacts
│   └── model
│       ├── MLmodel
│       ├── conda.yaml
│       ├── model.pkl
│       ├── python_env.yaml
│       └── requirements.txt
├── meta.yaml
├── metrics
│   ├── mae
│   ├── r2
│   └── rmse
├── params
│   ├── alpha
│   └── l1_ratio
└── tags
    ├── mlflow.log-model.history
    ├── mlflow.source.git.commit
    ├── mlflow.source.name
    ├── mlflow.source.type
    └── mlflow.user

5 directories, 16 files


In [ ]:
In particular, we are interested in the MLmodel file stored under artifacts/model:

In [26]:
!cat mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/MLmodel

artifact_path: model
flavors:
  python_function:
    env: conda.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    python_version: 3.7.4
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 0.20.3
mlflow_version: 1.28.0
model_uuid: 342c5d1852ff401eab8f492691f9c559
run_id: 280a5311590b4c5cb10b052cc6617819
utc_time_created: '2022-09-06 18:44:21.511028'


We can also view the dependencies that will be required to serve the model

In [29]:
!cat mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/conda.yaml

channels:
- conda-forge
dependencies:
- python=3.7.4
- pip<=22.1.2
- pip:
  - mlflow
  - cloudpickle==2.1.0
  - scikit-learn==0.20.3
name: mlflow-env


This file stores the details of how the model was stored. With this information (plus the other files in the folder), we are able to load the model back. Seldon’s MLflow server will use this information to serve this model.

### 3. Conda pack (optional)

In [5]:
conda pack -o mlruns/1/bffc73a34bab48ed9bb5f3ae17239cb2/artifacts/model/environment.tar.gz -f

Packing environment at '/home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/python3.8-mlflow-example' to 'mlruns/1/bffc73a34bab48ed9bb5f3ae17239cb2/artifacts/model/environment.tar.gz'
[########################################] | 100% Completed |  8.1s

Note: you may need to restart the kernel to use updated packages.


In [6]:
%%sh
ls mlruns/1/bffc73a34bab48ed9bb5f3ae17239cb2/artifacts/model/

MLmodel
conda.yaml
environment.tar.gz
model.pkl
python_env.yaml
requirements.txt


### 4. Push artefacts to MinIO

In [31]:
minioClient = Minio("35.204.34.216:9000", "admin", "F]L?yy!]&J%_-27t", secure=False)

In [32]:
minioClient.list_buckets()

[Bucket('aaai'),
 Bucket('aav-ilse'),
 Bucket('deploy-audit'),
 Bucket('jorik'),
 Bucket('jorik2'),
 Bucket('jorikmlflowexample'),
 Bucket('models'),
 Bucket('nnlp-mlflow-test'),
 Bucket('nnlptestn'),
 Bucket('pkl'),
 Bucket('tayfun')]

In [43]:
def upload_local_directory_to_minio(local_path: str, bucket_name: str, folder_name: str):
    assert os.path.isdir(local_path)

    for local_file in glob.glob(local_path + '/**'):
        local_file = local_file.replace(os.sep, "/")
        if not os.path.isfile(local_file):
            upload_local_directory_to_minio(
                local_file, bucket_name)
        else:
            remote_path = os.path.join(
                local_file[1 + len(local_path):])
            remote_path = remote_path.replace(
                os.sep, "/")
            remote_path = folder_name + "/" + remote_path
            minioClient.fput_object(bucket_name, remote_path, local_file)

In [44]:
upload_local_directory_to_minio("mlruns/0/280a5311590b4c5cb10b052cc6617819/artifacts/model", "models", "default")

### 5. Deploy via the SDK

In [11]:
SD_IP = "nn-group.cloud.seldon.io"

config = Configuration()
config.host = f"https://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"https://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = ">+HCA#AStf*NY$bD"
config.auth_method = "auth_code"

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

In [52]:
SD_IP = "nn-group.cloud.seldon.io"

config = Configuration()
config.host = f"https://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"https://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = ">+HCA#AStf*NY$bD"
config.auth_method = "client_credentials"

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

In [53]:
livenessProbe = {
    "initialDelaySeconds": 100,
    "failureThreshold": 300,
    "periodSeconds": 5,
    "successThreshold": 1,
    "httpGet": {
        "path": "/health/ping",
        "port": "http",
        "scheme": "HTTP"
    }
}
readinessProbe = {
    "initialDelaySeconds": 100,
    "failureThreshold": 300,
    "periodSeconds": 5,
    "successThreshold": 1,
    "httpGet": {
        "path": "/health/ping",
        "port": "http",
        "scheme": "HTTP"
    }
}

In [50]:
DEPLOYMENT_NAME = "wine-andrew"
NAMESPACE = "seldon-gitops"
MODEL_LOCATION = "minio:models/default"

PREPACKAGED_SERVER = "MLFLOW_SERVER"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": f"{DEPLOYMENT_NAME}",
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": f"{DEPLOYMENT_NAME}",
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol":"v2",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS,
                                            "nvidia.com/gpu":"0"
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS,
                                            "nvidia.com/gpu":"0"
                                        }
                                    },
                                    "livenessProbe": livenessProbe,
                                    "readinessProbe": readinessProbe
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "envSecretRefName": "minio-bucket",
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [ {
                            "name": "xtype",
                            "value": "DataFrame",
                            "type": "STRING"
                        }],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [51]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.list_seldon_deployments(namespace=NAMESPACE)

{'api_version': None,
 'items': [{'api_version': 'machinelearning.seldon.io/v1',
            'kind': 'SeldonDeployment',
            'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"machinelearning.seldon.io/v1alpha2","kind":"SeldonDeployment","metadata":{"annotations":{},"creationTimestamp":null,"labels":{"argocd.argoproj.io/instance":"seldon-gitops-seldon-gitops","fluentd":"true"},"name":"fraud","namespace":"seldon-gitops"},"spec":{"annotations":{"seldon.io/engine-seldon-log-messages-externally":"true"},"name":"fraud","predictors":[{"componentSpecs":[{"metadata":{"creationTimestamp":"2022-09-02T15:13:24Z"},"spec":{"containers":[{"name":"fraud-container","resources":{"limits":{"cpu":"1","memory":"1Gi"},"requests":{"cpu":"1","memory":"1Gi"}}}]}}],"engineResources":{},"graph":{"endpoint":{"type":"REST"},"implementation":"XGBOOST_SERVER","logger":{"mode":"all"},"modelUri":"gs://andrew-seldon/fraud-workshop/models/andrew","name":"fraud-conta

In [ ]:
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  protocol: v2  # Activate the v2 protocol
  name: wines
  predictors:
    - graph:
        children: []
        implementation: MLFLOW_SERVER
        modelUri: gs://seldon-models/test/elasticnet_wine_<uuid>
        name: classifier
      name: default
      replicas: 1

### 6. Run an inference request

In [ ]:
inference_request = {
    "parameters": {
        "content_type": "pd"
    },
    "inputs": [
        {
          "name": "fixed acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [7.4],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "volatile acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.7000],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "citric acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "residual sugar",
          "shape": [1],
          "datatype": "FP32",
          "data": [1.9],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "chlorides",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.076],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "free sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [11],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "total sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [34],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "density",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.9978],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "pH",
          "shape": [1],
          "datatype": "FP32",
          "data": [3.51],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "sulphates",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.56],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "alcohol",
          "shape": [1],
          "datatype": "FP32",
          "data": [9.4],
          "parameters": {
              "content_type": "np"
          }
        },
    ]
}

In [ ]:
endpoint = "http://localhost:8003/seldon/seldon/mlflow/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))